### Summarization and Q&A Using Transformer

- we need encoder-decoder architecture

- Encoder is for

- Decoder is for 

⸻

1. Understand the Transformer Architecture

At a high level, the original Transformer (from the 2017 paper) consists of:
	•	Input Embedding (plus positional encoding)
	•	Encoder Blocks (for encoding input)
	•	Decoder Blocks (for generating output)
	•	Multi-Head Self-Attention
	•	Feedforward Neural Networks
	•	Layer Normalization & Residual Connections

⸻

2. Minimal Setup (Start Here)

Use PyTorch or TensorFlow — most people use PyTorch for scratch implementations.

Basic Structure:

class Transformer(nn.Module):
    def __init__(self, ...):
        super().__init__()
        self.encoder = Encoder(...)
        self.decoder = Decoder(...)
    
    def forward(self, src, tgt):
        encoder_output = self.encoder(src)
        output = self.decoder(tgt, encoder_output)
        return output

⸻

3. Core Components to Implement
	•	Positional Encoding
	•	Multi-Head Attention
	•	Layer Normalization
	•	Feed-Forward Layer
	•	Encoder Layer (stacked N times)
	•	Decoder Layer (stacked N times)
	•	Final Linear + Softmax for prediction

⸻

4. Helpful Resources
	•	The Annotated Transformer (PyTorch):
	•	https://nlp.seas.harvard.edu/2018/04/03/attention.html
	•	Transformer from scratch in PyTorch (YouTube):
The AI Epiphany channel is gold for this.

⸻

5. Training Your Transformer

Once you build the model:
	•	Choose your dataset (e.g., for Q&A, use SQuAD; for summarization, try CNN/DailyMail).
	•	Tokenize your data.
	•	Define your loss (usually CrossEntropyLoss for text).
	•	Use teacher forcing during training.

⸻

Optional: Start Small

You can start by building:
	•	Just the Encoder (like BERT)
	•	Or a small Encoder-Decoder (like TinyT5)

⸻

## TODO:

- collect data:
    - collect all the ids
    - use https://arxiv.org/pdf/{id} to download pdf
    - read and save all the pdf articles in to csv file.

- which part of the transformer is controlling summarization vs translation?

In [62]:
# import os
# import kagglehub
# import pandas as pd
# import math

# Data Preparation

In [63]:
# # Download latest version
# path = kagglehub.dataset_download("Cornell-University/arxiv")

# print("Path to dataset files:", path)

In [64]:
# files = os.listdir(path)
# print(files)

In [65]:
# df = pd.read_json(path+'/arxiv-metadata-oai-snapshot.json', lines=True)
# print("First 5 records:", df.head())

# Model Architecture


In [66]:
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax
import copy
import math
import os
import pandas as pd

In [67]:
class EncoderDecoder(nn.Module):
    # TODO:
    # what is generator?
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    # what are these masks for why do we need two masks?
    def forward(self, src, tgt, src_mask, tgt_mask):
        # take in and process maksed src and tgt sequences
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [68]:
class Generator(nn.Module):
    '''
    Standard linear + softmax generation step
    '''
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        # TODO: which activation function is the best for summarization?
        return log_softmax(self.proj(x), dim=-1)

In [69]:
def clones(module, N):
    # Produce N identical layers
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [70]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        'pass the input (and mask) through each layer in turn'
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


In [71]:
class LayerNorm(nn.Module):
    '''Construct a layernorm module (See citation for details).'''
    # TODO: what is the purpose of eps?
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [72]:
# TODO: what is this function trying to do? what is the input and output?

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        # why add X to the tensor?
        return x + self.dropout(sublayer(self.norm(x)))

In [73]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)


In [74]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [75]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [76]:
def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)

    return subsequent_mask == 0

In [77]:
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = torch.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [78]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout = 0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        query, key, value = [
            l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for l, x in zip(self.linears, (query, key, value))
        ]

        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        x = (x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k))

        del query
        del key
        del value

        return self.linears[-1](x)


In [79]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [80]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [81]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [82]:
def make_model(src_vocab, tgt_vocab, N=6, d_model= 512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return model

In [83]:
def inference_test():
    test_model = make_model(11, 11, 2)
    test_model.eval()
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    memory = test_model.encode(src, src_mask)
    ys = torch.zeros(1, 1).type_as(src.data)

    for i in range(9):
        out = test_model.decode(memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data))
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    
    print("Example Untrained Model Prediction:", ys)

In [84]:
def run_tests():
    for _ in range (10):
        inference_test()

In [85]:
RUN_EXAMPLES = True

In [86]:
def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

In [87]:
show_example(run_tests)


Example Untrained Model Prediction: tensor([[0, 3, 4, 5, 8, 3, 4, 5, 8, 4]])
Example Untrained Model Prediction: tensor([[ 0,  1, 10,  1, 10,  1,  0,  5, 10,  1]])
Example Untrained Model Prediction: tensor([[ 0,  7, 10,  7,  2,  3,  7, 10,  7, 10]])
Example Untrained Model Prediction: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 8]])
Example Untrained Model Prediction: tensor([[0, 5, 3, 6, 4, 5, 8, 4, 5, 8]])
Example Untrained Model Prediction: tensor([[0, 9, 2, 2, 2, 2, 2, 2, 2, 2]])
Example Untrained Model Prediction: tensor([[0, 6, 7, 6, 7, 6, 7, 6, 7, 4]])
Example Untrained Model Prediction: tensor([[0, 5, 2, 1, 1, 1, 1, 1, 1, 1]])
Example Untrained Model Prediction: tensor([[ 0,  2, 10,  8,  2, 10,  8,  9,  6,  6]])
Example Untrained Model Prediction: tensor([[ 0,  6,  6,  6,  3,  6,  3,  3, 10,  2]])
